In [7]:
import mlflow

from mlflow.tracking import MlflowClient

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("C")

MLFLOW_TRACKING_URI = "http://localhost:5000"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [8]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='5',    # Experiment ID we want
    #filter_string="metrics.recall > 0.7",
    filter_string="metrics.f1 > 0.3",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    #order_by=["metrics.recall ASC"]
    order_by=["metrics.f1 ASC"]
)

In [9]:
for run in runs:
    print(f"run id: {run.info.run_id}, f1: {run.data.metrics['f1']:.4f}")

run id: de22006e68584f8b80632138d4ccd928, f1: 0.3478
run id: 533f3371b707417f8864efcd7d30d060, f1: 0.3478
run id: 0e308892254b405c90971e0aab2ef7aa, f1: 0.3478
run id: 04239d9115c645c683bb35054f9cd2a5, f1: 0.3478


In [10]:
runs[0].info.run_id

'de22006e68584f8b80632138d4ccd928'

In [11]:
logged_model = 'runs:/' + runs[0].info.run_id + '/models_mlflow'

print(logged_model)

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

runs:/de22006e68584f8b80632138d4ccd928/models_mlflow


In [12]:
import pickle
with open('./pickle/val.pkl', 'rb') as f_in:
    X_val,y_val = pickle.load(f_in)
print(X_val[0:10])

   etiv   nwbv  m/f  age  educ  ses
0  1402  0.822    0   60  18.0  1.0
1  1636  0.680    1   74  18.0  2.0
2  1549  0.730    0   71   4.0  1.0
3  1345  0.737    0   81  13.0  2.0
4  1832  0.769    1   76  16.0  3.0
5  1641  0.703    1   84   2.0  0.0
6  1508  0.805    0   67   3.0  4.0
7  1470  0.687    0   81  18.0  2.0
8  1307  0.679    1   90  12.0  3.0
9  1713  0.761    1   77   5.0  1.0


In [1]:
import pandas as pd
patient = {
    'm/f': 1,
    'age': 99,
    'educ':2.0,
    'ses': 0.0,
    'etiv': 1500,
    'nwbv': 0.1,    
}
X_patient = pd.DataFrame.from_dict([patient])
X_patient

,m/f,age,educ,ses,etiv,nwbv
0,1,99,2.0,0.0,1500,0.1


In [14]:
import numpy as np
# Predict on a Pandas DataFrame.
y_pred = loaded_model.predict(X_patient)

dementia_threshold = np.log1p(1.5 / 26)
y_pred = (y_pred > dementia_threshold).astype(int)
print(y_pred, y_val.to_numpy())

[1] [0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0]


In [ ]:
#mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

run_id = "70cc9d23610d4dd9969187cbfabb08a1"
model_uri = f"runs:/{run_id}/models"
mlflow.register_model(model_uri=model_uri, name="MRI_predictor")